In [1]:
import os
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import Tool, initialize_agent
from langchain.llms import OpenAI
import langchain
langchain.verbose = True
# LangChain Tracing
from langchain.callbacks import get_callback_manager, set_tracing_callback_manager
from langchain.callbacks.tracers import SharedLangChainTracer
set_tracing_callback_manager()

In [2]:
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

agent = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=True
)

In [3]:
tracer_session = SharedLangChainTracer().new_session()
print(f"New tracing session: {tracer_session}")

agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
)

POST http://127.0.0.1:5000/sessions, status code: 201, id: 1
New tracing session: TracerSession(id=1, start_time=datetime.datetime(2023, 1, 4, 1, 24, 12, 833822), extra={})


> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompts after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to answer questions about current events
Calculator: useful for when you need to answer questions about math

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Who won the US Open men's tennis final in 

"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."

In [4]:
tracer_session.to_dict()

{'id': 1,
 'start_time': datetime.datetime(2023, 1, 4, 1, 24, 12, 833822),
 'extra': {}}

In [5]:
import requests

r = requests.get("http://127.0.0.1:5000/sessions")
r.json()

[{'id': 1, 'start_time': '2023-01-04T01:24:12.833822', 'extra': {}}]

In [6]:
SharedLangChainTracer().load_session(1)

TracerSession(id=1, start_time=datetime.datetime(2023, 1, 4, 1, 24, 12, 833822, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST')), extra={})

In [7]:
SharedLangChainTracer().load_session(2) # Expect exception -- this session does not exist

TracerException: Failed to load session 2

In [8]:
r = requests.get("http://127.0.0.1:5000/chain-runs?id=1")
r.json()

{'id': 1,
 'start_time': 1672824253.114334,
 'end_time': 1672824269.433619,
 'extra': {},
 'execution_order': 1,
 'serialized': {'name': 'AgentExecutor'},
 'session_id': 1,
 'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
 'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
 'child_runs': [{'id': 4,
   'start_time': 1672824253.114394,
   'end_time': 1672824255.682817,
   'extra': {},
   'execution_order': 2,
   'serialized': {'name': 'LLMChain'},
   'session_id': 1,
   'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
    'agent_scratchpad': '',
    'stop': ['\nObservation: ']},
   'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: "US Open men\'s

In [9]:
r = requests.get("http://127.0.0.1:5000/llm-runs?top_level=True&session_id=1") # Should be empty -- no top level llm runs
r.json()

[]

In [10]:
r = requests.get("http://127.0.0.1:5000/tool-runs?session_id=2") # Should be empty -- nothing for session id 2
r.json()

[]

In [11]:
r = requests.get("http://127.0.0.1:5000/chain-runs")
print(f"{len(r.json())} total chain runs")
r.json()

7 total chain runs


[{'id': 1,
  'start_time': '2023-01-04T01:24:13.114334',
  'end_time': '2023-01-04T01:24:29.433619',
  'extra': {},
  'execution_order': 1,
  'serialized': {'name': 'AgentExecutor'},
  'session_id': 1,
  'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
  'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
  'child_runs': [{'id': 4,
    'start_time': '2023-01-04T01:24:13.114394',
    'end_time': '2023-01-04T01:24:15.682817',
    'extra': {},
    'execution_order': 2,
    'serialized': {'name': 'LLMChain'},
    'session_id': 1,
    'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
     'agent_scratchpad': '',
     'stop': ['\nObservation: ']},
    'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to 

In [12]:
r = requests.get("http://127.0.0.1:5000/chain-runs?top_level=True")
print(f"{len(r.json())} total top level runs")
r.json()

1 total top level runs


[{'id': 1,
  'start_time': '2023-01-04T01:24:13.114334',
  'end_time': '2023-01-04T01:24:29.433619',
  'extra': {},
  'execution_order': 1,
  'serialized': {'name': 'AgentExecutor'},
  'session_id': 1,
  'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
  'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
  'child_runs': [{'id': 4,
    'start_time': '2023-01-04T01:24:13.114394',
    'end_time': '2023-01-04T01:24:15.682817',
    'extra': {},
    'execution_order': 2,
    'serialized': {'name': 'LLMChain'},
    'session_id': 1,
    'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
     'agent_scratchpad': '',
     'stop': ['\nObservation: ']},
    'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to 

In [38]:
r = requests.get("http://127.0.0.1:5000/llm-runs?id=1")
print(r.json())

# Give feedback for llm run with id 1
feedback = {
    "id":None,
    "llm_run_id": 1,
    "chain_run_id": None,
    "binary_feedback": True,
    "text_feedback": "test",
    "label_feedback": "label",
    "score": 5
}
r = requests.post("http://127.0.0.1:5000/feedbacks", json=feedback)
print(r.json())

{'id': 1, 'start_time': 1672824267.369585, 'end_time': 1672824267.787714, 'extra': {}, 'execution_order': 13, 'serialized': {'name': 'OpenAI'}, 'session_id': 1, 'prompts': ["You are GPT-3, and you can't do math.\n\nYou can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.\n\nSo we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we’ll take care of the rest:\n\nQuestion: ${Question with hard calculation.}\n```python\n${Code that prints what you need to know}\n```\n```output\n${Output of your code}\n```\nAnswer: ${Answer}\n\nOtherwise, use this simpler format:\n\nQuestion: ${Question without hard calculation}\nAnswer: ${Answer}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n\n```python\nprint(37593 * 67)\n```\n```output\n2518731

In [41]:
# Get the feedback object just created for the llm run

r = requests.get("http://127.0.0.1:5000/feedbacks?llm_run_id=1")
print(r.json())

{'id': 1, 'llm_run_id': 1, 'chain_run_id': None, 'binary_feedback': True, 'text_feedback': 'test', 'label_feedback': 'label', 'score': 5, 'created_at': 1672825546.777361}


In [42]:
r = requests.get("http://127.0.0.1:5000/chain-runs?id=1")
print(r.json())

# Give feedback for llm run with id 1
feedback = {
    "id":None,
    "llm_run_id": None,
    "chain_run_id": 1,
    "binary_feedback": True,
    "text_feedback": "test",
    "label_feedback": "label",
    "score": 5
}
r = requests.post("http://127.0.0.1:5000/feedbacks", json=feedback)
print(r.json())

{'id': 1, 'start_time': 1672824253.114334, 'end_time': 1672824269.433619, 'extra': {}, 'execution_order': 1, 'serialized': {'name': 'AgentExecutor'}, 'session_id': 1, 'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"}, 'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."}, 'child_runs': [{'id': 4, 'start_time': 1672824253.114394, 'end_time': 1672824255.682817, 'extra': {}, 'execution_order': 2, 'serialized': {'name': 'LLMChain'}, 'session_id': 1, 'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?", 'agent_scratchpad': '', 'stop': ['\nObservation: ']}, 'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: "US Open men\'s tennis final 2019"'}, 'child_runs': [{'i

In [43]:
# Get all the feedbacks

r = requests.get("http://127.0.0.1:5000/feedbacks")
print(r.json())

[{'id': 1, 'llm_run_id': 1, 'chain_run_id': None, 'binary_feedback': True, 'text_feedback': 'test', 'label_feedback': 'label', 'score': 5, 'created_at': '2023-01-04T01:45:46.777361'}, {'id': 2, 'llm_run_id': None, 'chain_run_id': 1, 'binary_feedback': True, 'text_feedback': 'test', 'label_feedback': 'label', 'score': 5, 'created_at': '2023-01-04T01:48:44.340336'}]
